# In the Name of God
Name : Mohammad Mohajel Sadegi

SID  : 810199483


# Initial classes:

## DEFINES:

In [1]:
BFS_PRINT_PATH = 0

## Abstract:
Implementation of BFS, IDS and A* search 

BSF and IDS and A* search will give us optimal soloution.

As discussed in class, weighed-astar can find soloution faster but optimality is not guaranteed.

## Importing libraries:

In [2]:
import copy
from collections import Counter
from queue import Queue
import time
import numpy as np

## Class state

In [3]:
class State:
    def __init__(self, rocky_nodes) -> None:
        self.position = 0
        self.recipes_seen = [] #be very carefull changed
        self.morids_seen = []
        self.prev_state = None
        self.rocky_nodes = dict.fromkeys(rocky_nodes, 0)
        self.rocky_remain = 0
        self.cost = 0

    def __members(self):
        return (self.position, self.recipes_seen, self.morids_seen, self.rocky_remain) #delete self.cost


    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        return False

    def __hash__(self):
        return hash(f"{self.position},{self.recipes_seen},{self.morids_seen},{self.rocky_remain},{self.cost}")

    def __str__(self) -> str:
        return f"\n cost:{self.cost}, \
                 \n position{self.position + 1} \
                 \n morids:{[x+1 for x in self.morids_seen]} \
                 \n recipes:{[x+1 for x in self.recipes_seen]} \
                  \n\n"  

## Class graph:

In [4]:
class Graph:
    def __init__(self, v :int) -> None:
        self.v = v
        self.edges = [[] for _ in range(v)]
        self.morids = None #make it dictionary
        self.recipes = []
        self.start_position = 0
        self.rocky_nodes = []

    def add_edge(self, u :int, v :int) -> None:
        self.edges[u].append(v)
        self.edges[v].append(u)

    def is_rocky(self, node :int) -> bool:
        return (node in self.rocky_nodes)

    def is_recipes(self, node :int) -> bool:
        return (node in self.recipes)

    def is_morid(self, node :int) -> bool:
        return (node in list(self.morids.keys()))

    def get_neighbors(self, s :State):
        return self.edges[s.position]


## Read input and create graph class

In [5]:
def create_graph(path :str) -> Graph:
    file = open(path, "r")
    n, m = map(int, file.readline().split(" "))
    g = Graph(n)

    for _ in range(m):
        u, v = map(int, file.readline().split(" "))
        g.add_edge(u - 1, v - 1)

    h = int(file.readline())
    rocky_nodes = list(map(int, file.readline().split(" ")))
    rocky_nodes = [x - 1 for x in rocky_nodes]
    g.rocky_nodes = rocky_nodes
    s = int(file.readline())

    morids = {}
    for _ in range(s):
        line = list(map(int, file.readline().split(" ")))
        line = [x - 1 for x in line]
        morids.update({line[0] : line[2:]})
    g.morids = morids
    
    g.start_position = (int(file.readline()) - 1)

    recipes = []
    for recipe in morids.values():
        recipes.extend(recipe)

    recipes = list(dict.fromkeys(recipes)) # Remove Duplicates From a Python List 
    g.recipes = recipes
    return g

g1 = create_graph("input-easy.txt")
g2 = create_graph("input2-easy.txt")
g3 = create_graph("input3-easy.txt")
g4 = create_graph("input4.txt")


# Search Modules:


    Initial State:

create_initial_state function creates initial state which is the position which we start searching

note that there is a diffrence between states and nodes in graph that we whatch. we may be in same node but have diffrent states...

    Transition Modue:

transition(s :State, g :Graph, p :int) -> State function
gets one state and returns next-state

    Goal State:

when we deliver to all morids, we reach goal stata. reached_goal function computes that.
    

## Create initial state

In [6]:
def create_initial_state(g :Graph) -> State:
    s = State(g.rocky_nodes)
    s.position = g.start_position
    if s.position in g.recipes:
        s.recipes_seen.append(g.start_position)
    return s

initial_state1 = create_initial_state(g1)
initial_state2 = create_initial_state(g2)
initial_state3 = create_initial_state(g3)
initial_state4 = create_initial_state(g4)
 

## Transition module and goal state implementation:

In [7]:
counter = 0 # remember to initilize it to 0 in begenning

def reached_goal(s :State, g :Graph) -> bool:
    return len(g.morids) == len(s.morids_seen)

def is_in(p , q) -> bool:
    for i in q:
        if i not in p:
            return False
    return True
    
def transition(s :State, g :Graph, p :int) -> State:
    global counter
    counter += 1

    # if s.prev_state == None: #initial_state
    #     print(f"start : {s.position}\n")
    # else:
    #     print(f"{s.prev_state.position} => {s.position} \n")

    next_state = copy.deepcopy(s)
    next_state.prev_state = s
    next_state.cost += 1
    if s.rocky_remain != 0:
        next_state.rocky_remain -= 1
    else:
        next_state.position = p
        if g.is_rocky(p):
            next_state.rocky_remain = next_state.rocky_nodes[p]
            next_state.rocky_nodes[p] += 1

        if(g.is_recipes(p) and p not in s.recipes_seen):
            next_state.recipes_seen.append(p)

        if(g.is_morid(p) and p not in s.morids_seen and is_in(next_state.recipes_seen, g.morids[p])):
            next_state.morids_seen.append(p)
            
    # print(f"going to node {next_state.position + 1}, cost:{next_state.cost}")
    return next_state
    

## Print_path function:
to print path after finding it!

In [8]:
def print_path(final_state : State) -> None:
    print("PATH::")
    s = final_state
    while True:
        print(f"{s.position + 1} <== ", end = " ")
        s = s.prev_state
        if s.prev_state == None: #s is first node
            print(f"{s.position + 1} ")
            break
         

## Test function:

In [9]:
def test(g :Graph, method) -> None:
    global counter
    counter = 0
    start = time.time()
    path = method(g, create_initial_state(g))
    end = time.time()
    print(f"time:{end - start}")
    print_path(path)
    print(f"cost:{path.cost},")
    print(f"states:{counter},")

# BFS:

## BFS implementation:

we use a queue.in each turn, we dequeue from state and enqueue all succssors to queue until reaching goal state. becasu all path costs are same (1), we can get optimal soloution.

### With QUEUE:

In [10]:
def bfs(g :Graph, initial_state :State) -> State:
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.get()
        
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)
                if BFS_PRINT_PATH:
                    print(f"{t.prev_state.position + 1} => {t.position + 1}")
        if BFS_PRINT_PATH:
            print("\n")

### With List:

In [11]:
def bfs2(g :Graph, initial_state :State) -> State:
    q = []
    q.append(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.pop(0)
        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.append(t)
                explored.add(t)
                # print(f"{t.prev_state.position + 1} => {t.position + 1}")
        # print("\n")

### With List comprehension:

In [12]:
def h(s :State) -> int:
    return len(s.recipes_seen) + len(s.morids_seen)

def bfs4(g :Graph, initial_state :State) -> State:
    max_h = 0
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:

        while 1:
            state = q.get()

            if h(state) > max_h:
                max_h = h(state)
                break
            elif h(state) >= (max_h - 1):
                break

        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")
            
        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)

## BFS Test:

### Test1:

In [13]:
test(g1, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.02970123291015625
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:287,


PATH:

    8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 287 | 0.1 |

### Test2:

In [14]:
test(g2, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.05643057823181152
PATH::
8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 
cost:8,
states:551,


PATH:

    8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 551 | 0.4 |

### Test3:

In [15]:
test(g3, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
time:3.5810770988464355
PATH::
10 <==  11 <==  13 <==  1 <==  4 <==  9 <==  5 <==  12 <==  6 <==  2 <==  3 <==  10 <==  11 <==  13 
cost:13,
states:21442,


PATH:

    10 <==  11 <==  13 <==  1 <==  4 <==  9 <==  5 <==  12 <==  6 <==  2 <==  3 <==  10 <==  11 <==  13 

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 13 | 21442 | 12.4 |

### Test4:

In [16]:
test(g4, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.01444864273071289
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:92,


PATH:

    5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 92 | 0.03 |

# IDS:

## IDS implementation:

we use dfs with height 0,1,2,3,... til we reach goal state.

In [17]:
def dfs(state :State, step :int, g :Graph):
    if reached_goal(state, g):
        return state
    if step == 0:
        return None

    for neighbor in g.get_neighbors(state):
        next_state = transition(state, g, neighbor)
        result = dfs(next_state, step - 1, g)
        if result != None:
            return result
    return None

h = 0
def ids(g:Graph, initial_state :State) -> State:
    global h
    while True:
        h += 1
        print(f"h:{h}")

        result = dfs(initial_state, h, g)
        if result != None:
            return result
        print(f"cant find in h:{h}")

## IDS Tests:

### Test1:

In [18]:
h = 0
test(g1, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.38765740394592285
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:4632,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| IDS | 8 | 4632 | 0.5 |

### Test2:

In [19]:
h = 0
test(g2, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.9566226005554199
PATH::
8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 
cost:8,
states:10465,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| IDS | 8 | 10465 | 3.4 |

### Test3

In [20]:
h = 0
test(g3, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
cant find in h:8
h:9
cant find in h:9
h:10
cant find in h:10
h:11
cant find in h:11
h:12
cant find in h:12
h:13
time:91.28525066375732
PATH::
10 <==  11 <==  13 <==  1 <==  4 <==  9 <==  5 <==  12 <==  6 <==  2 <==  3 <==  10 <==  11 <==  13 
cost:13,
states:608362,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| IDS | 13 | 608362 | 90 |

### Test4:

In [21]:
h = 0
test(g4, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.04269599914550781
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:580,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| IDS | 8 | 580 | 0.2 |

# ASTAR:

## A-Star search: 

This huristic is consistent.

Because number of morids in any pass from initial state to goal state is consecutively increasing (better to say its non-decreasing).

So we choose -len(s.morids_seen) as our consistent huristic.

### Implementation:

we have two lists, explored and frountier.

we choose state s with minimum f(s) = h(s) + cost(s) from frontier and explore its successors.
    next_state = min(frountier, key = lambda s : huristic(s) + s.cost) 

and we remember explored states to make sure not to rewatch them.

it will give us optimal soloution if we make sure our huristic is consistent.

In [22]:
alpha = 1

def huristic(s :State) -> int:
    global alpha
    return -len(s.morids_seen) * alpha

def a_star(g:Graph, initial_state :State) -> State:
    explored = []
    frountier = [initial_state, ]
    cost = 0

    while True:
        next_state = min(frountier, key = lambda s : huristic(s) + s.cost)
        frountier.remove(next_state)
        explored.append(next_state)

        if next_state.cost > cost:
            cost = next_state.cost
            print(f"cost:{cost}")

        if reached_goal(next_state, g):
            return next_state
        for neighbor in g.get_neighbors(next_state):
            t = transition(next_state, g, neighbor)
            if t not in explored and t not in frountier:
                frountier.append(t)

## Test A-Star

In [23]:
alpha = 1

### Test1

In [24]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.008731603622436523
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:99,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| A-Star | 8 | 99 | 0.2 |

### Test2:

In [25]:
test(g2, a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.02321791648864746
PATH::
8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 
cost:8,
states:205,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| A-Star | 8 | 205 | 0.02 |

### Test3:

In [26]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
time:5.096969366073608
PATH::
10 <==  11 <==  13 <==  1 <==  4 <==  9 <==  5 <==  12 <==  6 <==  2 <==  3 <==  10 <==  11 <==  13 
cost:13,
states:5767,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| A-Star | 13 | 5767 | 16 |

### Test4:

In [27]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.007461071014404297
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:55,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| A-Star | 8 | 55 | 0.1 |

## Test Weighted A-Star1:
    let alpha = 5

In [28]:
alpha = 5

### Test1

In [29]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.008345603942871094
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:85,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:5 | 8 | 85 | 0.01 |

### Test2

In [30]:
test(g2 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.014543533325195312
PATH::
8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 
cost:8,
states:122,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:5 | 8 | 122 | 0.1 |

### Test3

In [31]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
time:0.1795032024383545
PATH::
11 <==  10 <==  5 <==  12 <==  5 <==  10 <==  5 <==  9 <==  4 <==  1 <==  4 <==  9 <==  2 <==  3 <==  10 <==  11 <==  13 
cost:16,
states:736,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:5 | 16 | 736 | 0.1 |

### Test4

In [32]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.006242990493774414
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:49,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:5 | 8 | 49 | 0.01 |

## Test Weighted A-Star2:
    let alpha = 10

In [33]:
alpha = 10

### Test1

In [34]:
test(g1 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.007957696914672852
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:85,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:10 | 8 | 85 | 0.01 |

### Test2

In [35]:
test(g2 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.010127544403076172
PATH::
8 <==  5 <==  7 <==  3 <==  12 <==  4 <==  9 <==  10 <==  9 
cost:8,
states:122,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:10 | 8 | 122 | 0.01 |

### Test3

In [36]:
test(g3 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
cost:9
cost:10
cost:11
cost:12
cost:13
cost:14
cost:15
cost:16
cost:17
time:0.05749154090881348
PATH::
11 <==  13 <==  1 <==  4 <==  9 <==  5 <==  12 <==  5 <==  10 <==  5 <==  9 <==  4 <==  9 <==  2 <==  3 <==  10 <==  11 <==  13 
cost:17,
states:283,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:10 | 17 | 283 | 0.08 |

### Test4

In [37]:
test(g4 , a_star)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
cost:8
time:0.003587961196899414
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:49,


|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| Alpha:10 | 8 | 49 | 0.01 |

# Conclusion:

    Number OF States Watched:

|     | Test1 | Test2 | Test3 | Test4 |
| --- |  ---  |  ---  |  ---  |  ---  |
| BFS |   287   |   551  |   21442  |  92 |
| IDS |    4632  | 10465  |   608362  | 580  |
| A-Star |   85   |  205   |  5767   |  55 |
| alpha:5 |   85   |   122  |   736  |  49 |
| alpha:10 |   85   |   122  |  283   |  49 |

As you can see, In general, if we compare speed of these methods:
    weighted A-star > A-star > BFS > IDS

BSF and IDS and A* search will give us optimal soloution.(A* is optimal if huristic is consistent)

As discussed previously, weighed-astar can find soloution faster but optimality is not guaranteed.others are optimal.

# EXTRA:

## PATH checker

In [38]:
path_test1 = [1, 3, 4, 5, 7, 10, 11, 9, 8]
path_test2 = [9, 10, 2, 4, 12, 3, 7, 5, 8]
path_test3 = [42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16]

def check_path(g :Graph, path) -> bool:
    print("all recipes:")
    print(g.recipes)
    print("all morids:")
    print(g.morids)
    state = create_initial_state(g)
    print(state)

    for i in path:
        state = transition(state, g, i-1)
        print(state)
    return reached_goal(state, g)

x = check_path(g2, path_test2)
print(x)

all recipes:
[2, 9, 3, 6]
all morids:
{7: [2, 9], 6: [9, 3, 6, 2]}

 cost:0,                  
 position9                  
 morids:[]                  
 recipes:[]                   



 cost:1,                  
 position9                  
 morids:[]                  
 recipes:[]                   



 cost:2,                  
 position10                  
 morids:[]                  
 recipes:[10]                   



 cost:3,                  
 position2                  
 morids:[]                  
 recipes:[10]                   



 cost:4,                  
 position4                  
 morids:[]                  
 recipes:[10, 4]                   



 cost:5,                  
 position12                  
 morids:[]                  
 recipes:[10, 4]                   



 cost:6,                  
 position3                  
 morids:[]                  
 recipes:[10, 4, 3]                   



 cost:7,                  
 position7                  
 morids:[7]        